In [35]:
import sagemaker
import boto3

import numpy as np

In [36]:
profile_name = 'crayon-site'

boto3.setup_default_session(profile_name=profile_name)

boto_session = boto3.Session(profile_name=profile_name) # specify your local aws profile
sagemaker_session = sagemaker.Session(boto_session)
SAGEMAKER_ROLE = "AmazonSageMaker-ExecutionRole-20200714T182988"

account_id = boto_session.client("sts").get_caller_identity().get("Account")
region_name = "us-east-2"
ecr_repository = "sitecli-inference-preprocessing-container-v0"
tag = ':latest'

processing_repository_uri = "{}.dkr.ecr.{}.amazonaws.com/{}".format(
    account_id, region_name, ecr_repository + tag
)

In [37]:
processing_repository_uri

'395166463292.dkr.ecr.us-east-2.amazonaws.com/sitecli-inference-preprocessing-container-v0:latest'

In [38]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

script_processor = ScriptProcessor(
    command=["python3"],
    image_uri=processing_repository_uri,
    role=SAGEMAKER_ROLE,
    instance_count=1,
    instance_type="ml.r5.large",
    sagemaker_session=sagemaker_session,
)

In [40]:
script_processor.run(
    code="postprocess.py",
    inputs=[
        ProcessingInput(
            source="s3://st-crayon-dev/cli/inference/2020-09-15-02-49-43-939113/output_crops",
            destination="/opt/ml/processing/input/data/",
        )
    ],
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/output/test/",
            destination="s3://st-crayon-dev/cli/inference/2020-09-15-02-49-43-939113/output_mask",
            s3_upload_mode="Continuous",
        ),
    ],
    arguments=[
        "crop_width",
        "896",
        "crop_height",
        "896",
        "input_width",
        "224",
        "input_height",
        "224",
        "n_classes",
        "6",
    ],
)


Job Name:  sitecli-inference-preprocessing-contain-2020-09-15-03-24-12-913
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://st-crayon-dev/cli/inference/2020-09-15-02-49-43-939113/output_crops', 'LocalPath': '/opt/ml/processing/input/data/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-395166463292/sitecli-inference-preprocessing-contain-2020-09-15-03-24-12-913/input/code/postprocess.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'output-1', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-2-395166463292/sitecli-inference-preprocessing-contain-2020-09-15-03-24-12-913/output/output-1', 'LocalPath': '/opt/ml/processing/output/test/', 'S3UploadMode': 'Continuous'}}]
..............